# Stochastic: CMA-ES

In [3]:
include("support_code.jl");

1 iteration CMA-ES 


In [4]:
using Distributions
using Random
p = let
	wheeler(x, a=1.5) = - exp(-(x[1]*x[2] - a)^2 -(x[2]-a)^2)
	f = x -> wheeler(x)
	xdomain = (0,3)
	ydomain = (0,3)

	Random.seed!(0)

	μ = [1.75,1.75]
	σ = 0.7
	Σ = [1.0 0.0; 0.0 1.0]

	#m: distribution mean 
	m = 4 + floor(Int, 3*log(length(μ)))
	m_elite = div(m,2)
	n = length(μ)

	#P: multivariate normal distribution
	P = MvNormal(μ, σ^2*Σ)
	xs = [rand(P) for i in 1 : m]
	ys = f.(xs)
	is = sortperm(ys) # best to worst

	
	ws = log((m+1)/2) .- log.(1:m) 
	ws[1:m_elite] ./= sum(ws[1:m_elite])

	μ_eff = 1 / sum(ws[1:m_elite].^2)
	cσ = (μ_eff + 2)/(n + μ_eff + 5)
	dσ = 1 + 2max(0, sqrt((μ_eff-1)/(n+1))-1) + cσ
	cΣ = (4 + μ_eff/n)/(n + 4 + 2μ_eff/n)
	c1 = 2/((n+1.3)^2 + μ_eff)
	cμ = min(1-c1, 2*(μ_eff-2+1/μ_eff)/((n+2)^2 + μ_eff))
	ws[m_elite+1:end] .*= -(1 + c1/cμ)/sum(ws[m_elite+1:end])

	δs = [(x - μ)/σ for x in xs]
	δw = sum(ws[i]*δs[is[i]] for i in 1 : m_elite)
	μ_cma = μ + σ*δw

	μ_ce = mean(xs[is[i]] for i in 1:m_elite)

	plots = Plots.Plot[]
	push!(plots, Plots.Image((x,y)->f([x,y]), xdomain, ydomain, colormap=pasteljet, colorbar=false, xbins=600, ybins=600))
	push!(plots, Plots.Contour(x->pdf(P, x), xdomain, ydomain, contour_style="draw color=white", style="white", xbins=151, ybins=151))
	push!(plots, Plots.Scatter([x[1] for x in xs], [x[2] for x in xs], style="mark=*, mark size=0.5, mark options={draw=white, fill=white}"))
	push!(plots, Plots.Scatter([μ_cma[1]], [μ_cma[2]], style="mark=*, mark size=1.5, mark options={draw=pastelBlue, fill=pastelBlue}"))
	push!(plots, Plots.Scatter([μ_ce[1]], [μ_ce[2]], style="mark=*, mark size=1.5, mark options={draw=pastelRed, fill=pastelRed}"))
	ax = Axis(plots, ymin=ydomain[1], xmin=xdomain[1], ymax=ydomain[2], xmax=xdomain[2], width="5.5cm", height="5.5cm", style="xtick=\\empty, ytick=\\empty, contour/labels=false, axis equal, view={0}{90}")
end
plot(p)

TikzPictures.TikzPicture("\\begin{axis}[\n  height = {5.5cm},\n  xmin = {0},\n  xmax = {3},\n  ymax = {3},\n  xtick=\\empty, ytick=\\empty, contour/labels=false, axis equal, view={0}{90},\n  ymin = {0},\n  width = {5.5cm},\n  enlargelimits = false,\n  axis on top\n]\n\n\\addplot[\n  point meta min = -0.9999937291166521,\n  point meta max = -3.924395857947463e-26\n] graphics[\n  xmin = 0,\n  xmax = 3,\n  ymin = 0,\n  ymax = 3\n] {tmp_10000000000001.png};\n\n\\addplot3[\n  contour prepared={draw color=white},\n  white\n] table {\n  2.179183536295812 3.0 0.05464580693753405\n  2.18 2.9997280709107175 0.05464580693753405\n  2.2 2.992741945283031 0.05464580693753405\n  2.22 2.9853028704703157 0.05464580693753405\n  2.2334894697219982 2.98 0.05464580693753405\n  2.24 2.9774770937482082 0.05464580693753405\n  2.26 2.9693486109352754 0.05464580693753405\n  2.28 2.960717158893954 0.05464580693753405\n  2.2815980508503406 2.96 0.05464580693753405\n  2.3 2.951830564663614 0.05464580693753405\n  2

Demo CMA-ES with flower function


In [5]:
using Distributions
using Random

p = let
    function flower(x; a=1, b=1, c=4)
        return a*norm(x) + b*sin(c*atan(x[2], x[1]))
    end
    f = x -> flower(x)
        xdomain = ( -3, 3)
		ydomain = ( -3, 3)
        
    G= GroupPlot(4,4,groupStyle="xlabels at=edge bottom, ylabels at =edge left, horizontal sep=0.5cm, vertical sep=0.5cm", style="xlabel=\$x_1\$, ylabel=\$x_2\$")


    function add_axis!(samples, P)
        plots = Plots.Plot[]
        push!(plots, Plots.Image((x,y)->f([x,y]), xdomain, ydomain, colormap=pasteljet, colorbar=false, xbins=600, ybins=600))
        push!(plots, Plots.Contour(x->pdf(P, x), xdomain, ydomain, contour_style="draw color=white", style="white", xbins=151, ybins=151))
        push!(plots, Plots.Scatter([x[1] for x in samples], [x[2] for x in samples], style="mark=*, mark size=0.5, mark options={draw=white, fill=white}"))
        ax = Axis(plots, ymin=ydomain[1], xmin=xdomain[1], ymax=ydomain[2], xmax=xdomain[2], width="5cm", height="5cm", style="xtick=\\empty, ytick=\\empty, contour/labels=false, axis equal, view={0}{90}")
        push!(G, ax)
    end

    Random.seed!(0)
    σ = 1.0
    x = [2.0, 2.0] # initial point
    μ = copy(x)    

    m = 4 + floor(Int, 3*log(length(μ)))
    m_elite = div(m,2)
    cm = 1.0

    n = length(x)
    w′s = log((m+1)/2) .- log.(1:m)
    w′s ./= sum(w′s[1:m_elite])

    μ_eff = 1 / sum(w′s[1:m_elite].^2)

    @assert 1 ≤ μ_eff ≤ m_elite

    cσ = (μ_eff + 2)/(n + μ_eff + 5)
    dσ = 1 + 2max(0, sqrt((μ_eff-1)/(n+1))-1) + cσ
    cΣ = (4 + μ_eff/n)/(n + 4 + 2μ_eff/n)
    c1 = 2/((n+1.3)^2 + μ_eff)
    cμ = min(1-c1, 2*(μ_eff-2+1/μ_eff)/((n+2)^2 + μ_eff))

    α1 = 1 + c1 / cμ
    α2 = 1 + 2μ_eff / (μ_eff + 2)
    α3 = (1 - c1 - cμ)/(n*cμ)
    α_low = min(α1, α2, α3)

    w′_pos =  sum(max(w′, 0) for w′ in w′s)
    w′_neg = -sum(min(w′, 0) for w′ in w′s)
    ws = [w′ ≥ 0 ? w′/w′_pos : α_low*w′/w′_neg for w′ in w′s]

    E_norm = n^0.5*(1-1/(4n)+1/(21*n^2))
    
    pσ = zeros(n)
    pΣ = zeros(n)
    Σ = Matrix(1.0I, n, n)
    μ = copy(x)


    @assert isapprox(sum(ws[1:m_elite]), 1.0, atol=1e-6)

    for k in 1 : prod(G.dimensions)
        P = MvNormal(μ, σ^2*Σ)
        xs = [rand(P) for i in 1 : m]
        ys = [f(x) for x in xs]
        is = sortperm(ys) # best to worst

        add_axis!(xs, P)

        # selection and mean update
        ys = [(x - μ)/σ for x in xs]
        yw = sum(ws[i]*ys[is[i]] for i in 1 : m_elite)
        μ += cm*σ*yw

        # step-size control
        C = Σ^-0.5
        pσ = (1-cσ)*pσ + sqrt(cσ*(2-cσ)*μ_eff)*C*yw
        σ *= exp(cσ/dσ * (norm(pσ)/E_norm - 1))

        # covariance adaptation
        hσ   = norm(pσ)/sqrt(1-(1-cσ)^(2k)) < (1.4 + 2/(n+1))*E_norm ? 1 : 0
        pΣ = (1-cΣ)*pΣ + hσ*sqrt(cΣ*(2-cΣ)*μ_eff)*yw
        w0 = [ws[i] ≥ 0 ? ws[i] : n*ws[i]/norm(C*ys[is[i]])^2 for i in 1 : m]
        Σ = (1-c1-cμ) * Σ + # regard old matrix # Eq. 47
            c1 * (pΣ*pΣ' + # plus rank one update
                  (1-hσ) * cΣ*(2-cΣ) * Σ) + # minor correction
            cμ * # plus rank mu update
                sum(w0[i]*ys[is[i]]*ys[is[i]]' for i in 1 : m)

        Σ = triu(Σ)+triu(Σ,1)' # enforce symmetry
    end
    G
end

GroupPlot(Axis[Axis(PGFPlots.Plots.Plot[PGFPlots.Plots.Image("tmp_10000000000002.png", -3, 3, -3, 3, -0.9606685031014937, 4.330122610536329, nothing, false, nothing, PGFPlots.ColorMaps.RGBArrayMap(RGB{Float64}[RGB(0.993248, 0.906157, 0.143936), RGB(0.983868, 0.904867, 0.136897), RGB(0.974417, 0.90359, 0.130215), RGB(0.964894, 0.902323, 0.123941), RGB(0.9553, 0.901065, 0.118128), RGB(0.945636, 0.899815, 0.112838), RGB(0.935904, 0.89857, 0.108131), RGB(0.926106, 0.89733, 0.104071), RGB(0.916242, 0.896091, 0.100717), RGB(0.906311, 0.894855, 0.098125)  …  RGB(0.277941, 0.056324, 0.381191), RGB(0.277018, 0.050344, 0.375715), RGB(0.276022, 0.044167, 0.370164), RGB(0.274952, 0.037752, 0.364543), RGB(0.273809, 0.031497, 0.358853), RGB(0.272594, 0.025563, 0.353093), RGB(0.271305, 0.019942, 0.347269), RGB(0.269944, 0.014625, 0.341379), RGB(0.26851, 0.009605, 0.335427), RGB(0.267004, 0.004874, 0.329415)], 0x00000000000001f4), nothing, nothing), PGFPlots.Plots.Contour([2.210334915491788e-12 2.6975